# AH ASSESSMENT | DATA ENGINEERING | TRANSFORM

In [1]:
import findspark
findspark.init()

In [2]:
import os
import shutil
import logging
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,to_date,regexp_replace,regexp_extract, expr
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType,DateType
from datetime import datetime

In [3]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

### INITIALISE SPARK SESSION

In [4]:
spark=SparkSession.builder.appName('AH_ASSESSMENT_LOAD').config("spark.jars.packages","com.crealytics:spark-excel_2.12:0.13.7").getOrCreate()

:: loading settings :: url = jar:file:/Users/nncube1/Library/Python/3.10/lib/python/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/nncube1/.ivy2/cache
The jars for the packages stored in: /Users/nncube1/.ivy2/jars
com.crealytics#spark-excel_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d7fb396e-daf6-4d41-bea1-2282901a2fba;1.0
	confs: [default]
	found com.crealytics#spark-excel_2.12;0.13.7 in central
	found org.apache.poi#poi;4.1.2 in central
	found commons-codec#commons-codec;1.13 in central
	found org.apache.commons#commons-collections4;4.4 in central
	found org.apache.commons#commons-math3;3.6.1 in central
	found com.zaxxer#SparseBitSet;1.2 in central
	found org.apache.poi#poi-ooxml;4.1.2 in central
	found org.apache.poi#poi-ooxml-schemas;4.1.2 in central
	found org.apache.xmlbeans#xmlbeans;3.1.0 in central
	found com.github.virtuald#curvesapi;1.06 in central
	found com.norbitltd#spoiwo_2.12;1.8.0 in central
	found org.scala-lang.modules#scala-xml_2.12;1.3.0 in central
	found com.github.pjfanning#excel-streaming-reader;2.3.6 in centr

In [5]:
spark

In [6]:
#PYSPARK
spark.conf.set("spark.sql.legacy.allowUntypedScalaUDF", "true")  # Equivalent to pd.set_option('mode.chained_assignment', None)
spark.conf.set("spark.sql.repl.eagerEval.enabled", "true")  # Equivalent to pd.set_option('display.max_rows', None)
spark.conf.set("spark.sql.repl.eagerEval.maxNumRows", 100)  # Equivalent to pd.set_option('display.max_columns', None)
spark.conf.set("spark.sql.repl.eagerEval.truncate", 0)
spark.conf.set("zeppelin.spark.printREPLOutput", "true")# Equivalent to pd.set_option('display.expand_frame_repr', False)

#PANDAS
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

### INGESTING THE FILE FROM THE TRANSFORMATION DATA ZONE 

##### For each table,create a source path based on the location of the parquet files

In [7]:
tables = ['visits', 'procedures', 'hospitals', 'owners', 'pets', 'fact_table']

In [8]:
#DEFINING LOCATIONS FOR FILE
source='2.TRANSFORMATION_LAYER'

In [9]:
table_sources = {}

for table in tables:
    table_sources[f'{table}_source'] = f'{source}/{table}/'
    print(table_sources[f'{table}_source'])
    

2.TRANSFORMATION_LAYER/visits/
2.TRANSFORMATION_LAYER/procedures/
2.TRANSFORMATION_LAYER/hospitals/
2.TRANSFORMATION_LAYER/owners/
2.TRANSFORMATION_LAYER/pets/
2.TRANSFORMATION_LAYER/fact_table/


##### Read in the files 

In [10]:
visits_path=table_sources['visits_source']
visits_partitioned = spark.read.parquet(visits_path)


23/10/11 23:31:40 WARN DataSource: Found duplicate column(s) in the data schema and the partition schema: `dt`


##### Declare db connection details

In [11]:
# Replacing with database connection details
database_url = "jdbc:postgresql://localhost:5432/vet_db"
database_properties = {
    "user": "postgres",
    "password": "",
    "driver": "org.postgresql.Driver"
}


##### Write to db.I couldnt install the postgres driver(admin permissions) on my mac,hence the fail

In [12]:
try:
    visits_partitioned.write \
        .mode("overwrite") \
        .jdbc(url=database_url, table="visits", properties=database_properties)
except Exception as e:
    print(f"Error: {e}")


Error: An error occurred while calling o41.jdbc.
: java.lang.ClassNotFoundException: org.postgresql.Driver
	at java.net.URLClassLoader.findClass(URLClassLoader.java:382)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:424)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:357)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:101)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:101)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:101)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:218)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:222)
	at org.apache.spark.sql.execution.datasource